In [87]:
import cudf, cugraph
print(cudf.__version__, cugraph.__version__)

25.02.02 25.02.00


In [88]:
import pandas as pd
import os

BASE_DIR = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data"
POPULATION_PATH = os.path.join(BASE_DIR, "population.json")
OUTPUT_POPULATION_PATH = os.path.join(BASE_DIR, "population_corrected.json")

pop_df = pd.read_json(POPULATION_PATH)
pop_df = pop_df.rename(columns={'District': 'LIE_NAME'})
print(f"Initial population total rows: {len(pop_df)}")
print(f"Initial population unique LIE_NAME: {pop_df['LIE_NAME'].nunique()}")
print(f"Initial population duplicates: {pop_df['LIE_NAME'].duplicated().sum()}")
if pop_df['LIE_NAME'].duplicated().sum() > 0:
    print("Population duplicate LIE_NAME details:")
    print(pop_df[pop_df['LIE_NAME'].duplicated(keep=False)][['LIE_NAME']].reset_index().to_string(index=False))

pop_df.loc[55, 'LIE_NAME'] = '北投中央里'
pop_df.loc[377, 'LIE_NAME'] = '中山中央里'
pop_df.loc[208, 'LIE_NAME'] = '士林新安里'
pop_df.loc[443, 'LIE_NAME'] = '萬華新安里'

print(f"\nAfter population correction:")
print(f"Total rows: {len(pop_df)}")
print(f"Unique LIE_NAME: {pop_df['LIE_NAME'].nunique()}")
print(f"Duplicates: {pop_df['LIE_NAME'].duplicated().sum()}")

pop_df.to_json(OUTPUT_POPULATION_PATH, orient='records', force_ascii=False)
print(f"Corrected population JSON saved to: {OUTPUT_POPULATION_PATH}")

Initial population total rows: 456
Initial population unique LIE_NAME: 454
Initial population duplicates: 2
Population duplicate LIE_NAME details:
 index LIE_NAME
    55      中央里
   208      新安里
   377      中央里
   443      新安里

After population correction:
Total rows: 456
Unique LIE_NAME: 456
Duplicates: 0
Corrected population JSON saved to: /home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/population_corrected.json


Cell 1: Imports ,Global Constants

In [89]:
import geopandas as gpd
import pandas as pd
import cudf
import cugraph
from tqdm import tqdm
import logging
import os
import numpy as np
from scipy.stats import pearsonr
from shapely import make_valid
from shapely.errors import GEOSException
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from IPython.display import Image, display
from sklearn.cluster import KMeans
import plotly.express as px  
import plotly.graph_objects as go  
import cupy
import hashlib
import json
import networkx as nx
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv, BatchNorm
from keplergl import KeplerGl

try:
    import cuspatial
    CUSPATIAL_AVAILABLE = False
except ImportError:
    logging.warning("CuSpatial not available, falling back to CPU-based computation.")
    CUSPATIAL_AVAILABLE = False

%matplotlib inline

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Noto Sans CJK TC', 'Noto Serif CJK TC', 'Noto Sans Mono CJK TC', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

BASE_DIR = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data"
LANDUSE_NDVI_PATH = os.path.join(BASE_DIR, "neighborhoods_with_ndvi_numerical_corrected.geojson")
OSM_BUILDINGS_PATH = os.path.join(BASE_DIR, "Taipei_Buildings_fulldata.geojson")
OSM_ROADS_PATH = os.path.join(BASE_DIR, "taipei_segments_cleaned_verified.geoparquet")
OSM_TREES_PATH = os.path.join(BASE_DIR, "taipei_land.geoparquet")
OSM_TRANSIT_PATH = os.path.join(BASE_DIR, "taipei_infrastructure.geoparquet")
URBAN_MASTERPLAN_PATH = os.path.join(BASE_DIR, "Taipei_urban_masterplan.geojson")
ACCIDENTS_PATH = os.path.join(BASE_DIR, "2023_accidents.geojson")
POPULATION_PATH = os.path.join(BASE_DIR, "population_corrected.json")
SUBGRAPH_DIR = os.path.join(BASE_DIR, "subgraphs")
CHECKPOINT_DIR = os.path.join(BASE_DIR, "checkpoints")
INTERSECTION_CACHE_PATH = os.path.join(BASE_DIR, "neighborhoods_with_intersections.geoparquet")
GRAPH_NODES_CACHE_PATH = os.path.join(BASE_DIR, "graph_nodes.parquet")
GRAPH_EDGES_CACHE_PATH = os.path.join(BASE_DIR, "graph_edges.parquet")
GRAPH_NODE_ID_CACHE_PATH = os.path.join(BASE_DIR, "graph_node_id_to_index.json")
GRAPH_DATA_HASH_PATH = os.path.join(BASE_DIR, "graph_data_hash.txt")

os.makedirs(SUBGRAPH_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

CATEGORY_PRIORITY = {
    'City_Open_Area': 10,
    'Pedestrian': 9,
    'Public_Transportation': 8,
    'Amenity': 7,
    'Education': 6,
    'Medical': 5,
    'Commercial': 4,
    'Residential': 3,
    'Natural': 2,
    'Road': 1,
    'River': 1,
    'Infrastructure': 1,
    'Government': 1,
    'Special_Zone': 1,
    'Military': 1,
    'Industrial': 1,
    'Agriculture': 1
}

land_use_weights = {
    'city_open_area': 0.8,
    'commercial': 0.7,
    'infrastructure': 0.4,
    'government': 0.5,
    'public_transportation': 0.8,
    'education': 0.7,
    'medical': 0.6,
    'amenity': 0.8,
    'road': 0.3,
    'pedestrian': 1.0,
    'natural': 0.7,
    'special_zone': 0.4,
    'river': 0.7,
    'military': 0.2,
    'residential': 0.6,
    'industrial': 0.3,
    'agriculture': 0.4
}

Cell 2: Utility Functions

In [90]:
def print_data_structure(data_dict):
    print("\n--- Detailed Data Structure Overview ---")
    for key, df in data_dict.items():
        if isinstance(df, (gpd.GeoDataFrame, pd.DataFrame, cudf.DataFrame)):
            df = df.to_pandas() if isinstance(df, cudf.DataFrame) else df
            print(f"\nDataset: {key}")
            print(f"Shape: {df.shape}")
            print(f"Columns: {list(df.columns)}")
            print(f"Data types:\n{df.dtypes}")
            print(f"Missing values per column:\n{df.isnull().sum()}")
            if not df.select_dtypes(include=['float64', 'int64']).empty:
                summary = df.describe().round(2)
                print(f"Summary statistics for numerical columns:\n{summary}")
            for col in df.columns:
                if col in ['LIE_NAME', 'vertex', 'type', 'class', 'building', 'Category']:
                    unique_count = df[col].nunique()
                    print(f"Unique values in {col}: {unique_count}")
                    duplicates = df[col].duplicated().sum()
                    if duplicates > 0:
                        print(f"Duplicates in {col}: {duplicates}")
                        print(f"Sample duplicates in {col}:\n{df[df[col].duplicated(keep=False)][col].head()}")
                    if col == 'class' and key == 'roads':
                        print(f"Value counts for {col}:\n{df[col].value_counts()}")
            print("Sample data (first 5 rows):")
            sample = df.head(5).copy()
            for col in sample.select_dtypes(include=['float64', 'int64']).columns:
                sample[col] = sample[col].round(2)
            print(sample)
    print("--- End of Detailed Data Structure Overview ---\n")

def fix_geometry(geom, buffer_size=1e-5):
    if geom is None or geom.is_empty:
        return geom
    geom = make_valid(geom)
    if not geom.is_valid:
        geom = geom.buffer(buffer_size)
        geom = make_valid(geom)
    if not geom.is_valid:
        logging.warning(f"Geometry remains invalid after fixing: {geom.bounds}")
    return geom

def print_percentage_calculation(neighborhoods_gdf, urban_masterplan_gdf, sample_size=3):
    print("\n--- Percentage Calculation Process ---")
    sample_neighborhoods = neighborhoods_gdf.sample(min(sample_size, len(neighborhoods_gdf)), random_state=42)
    
    for idx, row in sample_neighborhoods.iterrows():
        lie_name = row['LIE_NAME']
        print(f"\nNeighborhood: {lie_name} (Index: {idx})")
        
        neighborhood_geom = fix_geometry(row['geometry'])
        if not neighborhood_geom.is_valid:
            print(f"Neighborhood geometry is invalid after fixing: {lie_name}")
            continue
        
        relevant_masterplan = urban_masterplan_gdf[urban_masterplan_gdf.intersects(neighborhood_geom)]
        if relevant_masterplan.empty:
            print("No master plan polygons intersect with this neighborhood.")
            continue
        
        temp_gdf = gpd.GeoDataFrame({'geometry': [neighborhood_geom]}, crs='EPSG:3826')
        intersected = gpd.overlay(temp_gdf, relevant_masterplan, how='intersection', keep_geom_type=False)
        if intersected.empty:
            print("No valid intersections after overlay.")
            continue
        
        intersected['geometry'] = intersected['geometry'].apply(fix_geometry)
        intersected = intersected[intersected.geometry.is_valid & ~intersected.geometry.is_empty]
        if intersected.empty:
            print("No valid geometries after fixing intersected polygons.")
            continue
        
        intersected['priority'] = intersected['Category'].map(CATEGORY_PRIORITY)
        intersected = intersected.sort_values(by='priority', ascending=False)
        
        total_area_geom = intersected.geometry.union_all()
        total_area_geom = fix_geometry(total_area_geom)
        if not total_area_geom.is_valid or total_area_geom.is_empty:
            print("Total area geometry is invalid after fixing.")
            continue
        total_area = total_area_geom.area
        print(f"Total unique master plan area: {total_area:.2f} m²")
        
        remaining_geom = total_area_geom
        category_areas = {}
        unique_categories = intersected['Category'].unique()
        
        for category in sorted(unique_categories, key=lambda x: CATEGORY_PRIORITY.get(x, 0), reverse=True):
            category_rows = intersected[intersected['Category'] == category]
            category_geom = category_rows.geometry.union_all()
            category_geom = fix_geometry(category_geom)
            if not category_geom.is_valid or category_geom.is_empty:
                print(f"Geometry for category {category} is invalid after fixing.")
                category_areas[category] = 0.0
                continue
            
            try:
                category_area_geom = category_geom.intersection(remaining_geom)
                category_area_geom = fix_geometry(category_area_geom)
                if not category_area_geom.is_valid or category_area_geom.is_empty:
                    print(f"Intersection geometry for category {category} is invalid after fixing.")
                    category_areas[category] = 0.0
                    continue
                
                category_area = category_area_geom.area
                category_areas[category] = category_area
                print(f"Area of {category} (priority {CATEGORY_PRIORITY.get(category, 0)}): {category_area:.2f} m²")
                
                remaining_geom = remaining_geom.difference(category_area_geom)
                remaining_geom = fix_geometry(remaining_geom)
                if not remaining_geom.is_valid or remaining_geom.is_empty:
                    print(f"Remaining geometry is invalid after subtracting {category}.")
                    break
            except GEOSException as e:
                print(f"Topology error for category {category}: {e}")
                category_areas[category] = 0.0
                continue
        
        print("\nPercentages:")
        total_percentage = 0.0
        for category, area in category_areas.items():
            percentage = (area / total_area * 100) if total_area > 0 else 0.0
            total_percentage += percentage
            print(f"{category}: {percentage:.2f}%")
        print(f"Sum of percentages: {total_percentage:.2f}%")
    print("--- End of Percentage Calculation Process ---\n")
    
def compute_data_hash(data_dict):
    hasher = hashlib.sha256()
    
    for key, df in data_dict.items():
        if isinstance(df, (gpd.GeoDataFrame, pd.DataFrame, cudf.DataFrame)):
            df = df.to_pandas() if isinstance(df, cudf.DataFrame) else df
            hasher.update(str(df.shape).encode('utf-8'))
            hasher.update(str(sorted(df.columns)).encode('utf-8'))
            sample = df.head(5).to_json()
            hasher.update(sample.encode('utf-8'))
    
    return hasher.hexdigest()

def compute_road_type_accident_correlation(roads_gdf, neighborhoods_gdf, accidents_gdf):
    """
    Compute correlation between OSM road types and accident density (accidents per km of road length).
    Uses road class as a proxy for width, with ordinal ranking based on OSM hierarchy.
    Generates bar, box, and scatter plots for visualization.
    """
    logging.info("Computing correlation between road types and accident density...")
    
    # Define ordinal width proxy based on OSM highway hierarchy
    width_ranking = {
        'motorway': 5,
        'trunk': 5,
        'primary': 4,
        'secondary': 4,
        'tertiary': 3,
        'residential': 3,
        'living_street': 3,
        'service': 2,
        'track': 2,
        'path': 1,
        'footway': 1,
        'cycleway': 1,
        'steps': 1,
        'pedestrian': 1,
        'unclassified': 0,
        'bridleway': 0,
        'unknown': 0
    }
    
    # Assign width rank to roads
    roads_gdf = roads_gdf.copy()
    roads_gdf['width_rank'] = roads_gdf['class'].map(width_ranking).fillna(0).astype(int)
    
    # Assign accidents to the nearest road using sjoin_nearest
    logging.info("Assigning accidents to nearest road...")
    joined_gdf = gpd.sjoin_nearest(accidents_gdf, roads_gdf, how='left', distance_col='distance')
    
    # Count accidents per road segment
    accident_counts = joined_gdf.groupby(joined_gdf.index_right).size().reindex(roads_gdf.index, fill_value=0)
    roads_gdf['accident_count'] = accident_counts
    
    # Filter short roads to avoid inflated accident density
    roads_gdf = roads_gdf[roads_gdf['length_m'] >= 10]
    
    # Compute accident density (accidents per km)
    roads_gdf['accident_density'] = roads_gdf['accident_count'] / (roads_gdf['length_m'] / 1000)
    roads_gdf['accident_density'] = roads_gdf['accident_density'].fillna(0).replace([np.inf, -np.inf], 0)
    
    # Log road type counts
    logging.info(f"Road type counts:\n{roads_gdf['class'].value_counts()}")
    print(f"Road type counts:\n{roads_gdf['class'].value_counts()}")
    
    # Aggregate by road class for summary
    summary = roads_gdf.groupby('class').agg({
        'length_m': 'sum',
        'accident_count': 'sum',
        'accident_density': 'mean',
        'width_rank': 'first'
    }).reset_index()
    
    # Filter out classes with negligible data
    summary = summary[summary['length_m'] > 1000]  # At least 1km total length
    summary = summary[summary['width_rank'] > 0]   # Exclude unknown, unclassified, bridleway
    
    # Log summary
    print("\n--- Road Type Accident Density Summary ---")
    print(summary[['class', 'length_m', 'accident_count', 'accident_density', 'width_rank']].round(2))
    
    # Compute Spearman's rank correlation
    from scipy.stats import spearmanr
    if len(summary) >= 2:
        corr, p_value = spearmanr(summary['width_rank'], summary['accident_density'])
        logging.info(f"Spearman's correlation between road width rank and accident density: {corr:.3f} (p-value: {p_value:.3f})")
        print(f"Spearman's correlation: {corr:.3f} (p-value: {p_value:.3f})")
    else:
        logging.warning("Insufficient road types for correlation analysis.")
        print("Insufficient road types for correlation analysis.")
    
    # Compute average accident density per neighborhood for walkability
    logging.info("Computing average road accident density per neighborhood...")
    road_neighborhoods = gpd.sjoin(roads_gdf[['geometry', 'class', 'length_m', 'width_rank', 'accident_density']], 
                                   neighborhoods_gdf[['geometry', 'LIE_NAME']], 
                                   how='left', predicate='intersects')
    avg_accident_density = road_neighborhoods.groupby('LIE_NAME')['accident_density'].mean().reindex(neighborhoods_gdf['LIE_NAME'], fill_value=0)
    neighborhoods_gdf['avg_road_accident_density'] = avg_accident_density
    
    # Visualize with multiple charts
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # 1. Bar Chart: Mean accident density by road type
    plt.figure(figsize=(12, 6))
    summary_sorted = summary.sort_values('width_rank', ascending=False)
    sns.barplot(data=summary_sorted, x='class', y='accident_density', hue='width_rank', dodge=False)
    plt.xlabel('Road Type')
    plt.ylabel('Mean Accident Density (Accidents per km)')
    plt.title('Mean Accident Density by Road Type')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Width Rank')
    plt.tight_layout()
    bar_path = os.path.join(BASE_DIR, 'road_type_accident_bar.png')
    plt.savefig(bar_path)
    plt.close()
    logging.info(f"Bar chart saved to {bar_path}")
    print(f"Bar chart saved to {bar_path}")
    
    # 2. Box Plot: Distribution of accident density by road type
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=roads_gdf[roads_gdf['class'].isin(summary['class'])], 
                x='class', y='accident_density', hue='width_rank', dodge=False)
    plt.xlabel('Road Type')
    plt.ylabel('Accident Density (Accidents per km)')
    plt.title('Distribution of Accident Density by Road Type')
    plt.xticks(rotation=45, ha='right')
    plt.yscale('log')  # Log scale for skewed data
    plt.legend(title='Width Rank')
    plt.tight_layout()
    box_path = os.path.join(BASE_DIR, 'road_type_accident_box.png')
    plt.savefig(box_path)
    plt.close()
    logging.info(f"Box chart saved to {box_path}")
    print(f"Box chart saved to {box_path}")
    
    # 3. Scatter Plot: Accident density vs. width rank with trend line
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=summary, x='width_rank', y='accident_density', 
                    size='length_m', sizes=(50, 500), hue='class', style='class', alpha=0.7)
    # Add trend line
    z = np.polyfit(summary['width_rank'], summary['accident_density'], 1)
    p = np.poly1d(z)
    plt.plot(summary['width_rank'], p(summary['width_rank']), "r--", alpha=0.5)
    plt.xlabel('Road Width Rank (1=Path, 5=Motorway)')
    plt.ylabel('Mean Accident Density (Accidents per km)')
    plt.title('Road Type vs. Accident Density')
    plt.yscale('log')  # Log scale for skewed data
    plt.grid(True)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    scatter_path = os.path.join(BASE_DIR, 'road_type_accident_scatter.png')
    plt.savefig(scatter_path)
    plt.close()
    logging.info(f"Scatter plot saved to {scatter_path}")
    print(f"Scatter plot saved to {scatter_path}")
    
    # Log top accident-prone road types
    top_types = summary.nlargest(3, 'accident_density')[['class', 'accident_density']]
    logging.info(f"Top 3 road types by accident density:\n{top_types.round(2)}")
    print(f"Top 3 road types by accident density:\n{top_types.round(2)}")
    
    return summary

Cell 3: Walkability Computation Functions

In [91]:
def compute_walkability_components(neighborhoods_gdf, sample_size=5):
    sample_gdf = neighborhoods_gdf.sample(min(sample_size, len(neighborhoods_gdf)), random_state=42)
    
    components = {
        'LIE_NAME': [],
        'land_use_diversity': [],
        'green_space_score': [],
        'transit_score': [],
        'road_connectivity': [],
        'safety_score': [],
        'elderly_accessibility': [],
        'walkability_score': []
    }
    
    ndvi_min, ndvi_max = neighborhoods_gdf['ndvi_mean'].min(), neighborhoods_gdf['ndvi_mean'].max()
    tree_min, tree_max = neighborhoods_gdf['tree_count'].min(), neighborhoods_gdf['tree_count'].max()
    transit_min, transit_max = neighborhoods_gdf['transit_count'].min(), neighborhoods_gdf['transit_count'].max()
    
    if 'intersection_density' in neighborhoods_gdf.columns:
        intersection_density_min = neighborhoods_gdf['intersection_density'].min()
        intersection_density_max = neighborhoods_gdf['intersection_density'].max()
    else:
        logging.warning("'intersection_density' column missing in neighborhoods_gdf. Defaulting to 0.")
        intersection_density_min = 0
        intersection_density_max = 1
    
    # For accident_count
    accident_count_min, accident_count_max = neighborhoods_gdf['accident_count'].min(), neighborhoods_gdf['accident_count'].max()
    # For avg_road_accident_density
    if 'avg_road_accident_density' in neighborhoods_gdf.columns:
        accident_density_max = neighborhoods_gdf['avg_road_accident_density'].max()
    else:
        logging.warning("'avg_road_accident_density' column missing. Defaulting to 0.")
        accident_density_max = 1
    
    for idx, row in sample_gdf.iterrows():
        land_use_cols = [f"land_use_{category.lower()}_percent" for category in CATEGORY_PRIORITY.keys()]
        land_use_values = [row.get(col, 0.0) / 100 for col in land_use_cols if col in row]
        land_use_values = [v for v in land_use_values if v > 0]
        if land_use_values:
            entropy = -np.sum([p * np.log2(p) for p in land_use_values])
            max_entropy = np.log2(len(land_use_values)) if len(land_use_values) > 0 else 1
            land_use_diversity = entropy / max_entropy if max_entropy > 0 else 0
        else:
            land_use_diversity = 0
        
        ndvi_normalized = ((row['ndvi_mean'] - ndvi_min) / (ndvi_max - ndvi_min + 1e-6))
        tree_normalized = ((row['tree_count'] - tree_min) / (tree_max - tree_min + 1e-6))
        open_area = row.get('land_use_city_open_area_percent', 0.0) / 100
        green_space_score = (ndvi_normalized + tree_normalized + open_area) / 3
        
        transit_raw = (row['transit_count'] - transit_min) / (transit_max - transit_min + 1e-6)
        transit_score = np.log1p(transit_raw * 10) / np.log1p(10)
        
        intersection_density = row.get('intersection_density', 0.0)
        intersection_density_normalized = (intersection_density - intersection_density_min) / (intersection_density_max - intersection_density_min + 1e-6)
        road_connectivity = np.log1p(intersection_density_normalized * 10) / np.log1p(10)
        
        # Hybrid safety score: combine accident_count and avg_road_accident_density
        accident_count_density = row['accident_count'] / row['area_km2'] if row['area_km2'] > 0 else 0
        accident_count_density_max = (accident_count_max / neighborhoods_gdf['area_km2'].min()) if neighborhoods_gdf['area_km2'].min() > 0 else 1
        accident_count_density = min(accident_count_density, accident_count_density_max * 0.5)  # Cap to reduce urban bias
        safety_score_count = 1 - (accident_count_density / (accident_count_density_max + 1e-6))
        
        accident_density = row.get('avg_road_accident_density', 0.0)
        safety_score_roads = 1 - (accident_density / (accident_density_max + 1e-6))
        
        # Combine both safety metrics (50% weight each)
        safety_score = 0.5 * safety_score_count + 0.5 * safety_score_roads
        
        elderly_accessibility = row['elderly_percentage'] / 100
        
        base_score = (
            0.3 * land_use_diversity +
            0.3 * green_space_score +
            0.2 * transit_score +
            0.2 * road_connectivity
        )
        
        safety_modifier = 0.3 + 0.7 * safety_score  # Increased weight for safety
        elderly_modifier = 1 + elderly_accessibility
        walkability_score = base_score * safety_modifier * elderly_modifier
        
        walkability_score = 1 / (1 + np.exp(-5 * (walkability_score - 1)))
        
        components['LIE_NAME'].append(row['LIE_NAME'])
        components['land_use_diversity'].append(land_use_diversity)
        components['green_space_score'].append(green_space_score)
        components['transit_score'].append(transit_score)
        components['road_connectivity'].append(road_connectivity)
        components['safety_score'].append(safety_score)
        components['elderly_accessibility'].append(elderly_accessibility)
        components['walkability_score'].append(walkability_score)
    
    return pd.DataFrame(components)

def compute_walkability_components_all(neighborhoods_df):
    components = {
        'LIE_NAME': [],
        'land_use_diversity': [],
        'green_space_score': [],
        'transit_score': [],
        'road_connectivity': [],
        'safety_score': [],
        'elderly_accessibility': [],
        'walkability_score': []
    }
    
    ndvi_min, ndvi_max = neighborhoods_df['ndvi_mean'].min(), neighborhoods_df['ndvi_mean'].max()
    tree_min, tree_max = neighborhoods_df['tree_count'].min(), neighborhoods_df['tree_count'].max()
    transit_min, transit_max = neighborhoods_df['transit_count'].min(), neighborhoods_df['transit_count'].max()
    
    if 'intersection_density' in neighborhoods_df.columns:
        intersection_density_min = neighborhoods_df['intersection_density'].min()
        intersection_density_max = neighborhoods_df['intersection_density'].max()
    else:
        logging.warning("'intersection_density' column missing. Defaulting to 0.")
        intersection_density_min = 0
        intersection_density_max = 1
    
    # For accident_count
    accident_count_min, accident_count_max = neighborhoods_df['accident_count'].min(), neighborhoods_df['accident_count'].max()
    # For avg_road_accident_density
    if 'avg_road_accident_density' in neighborhoods_df.columns:
        accident_density_max = neighborhoods_df['avg_road_accident_density'].max()
    else:
        logging.warning("'avg_road_accident_density' column missing. Defaulting to 0.")
        accident_density_max = 1
    
    for idx, row in neighborhoods_df.iterrows():
        land_use_cols = [f"land_use_{category.lower()}_percent" for category in CATEGORY_PRIORITY.keys()]
        land_use_values = [row.get(col, 0.0) / 100 for col in land_use_cols if col in row]
        land_use_values = [v for v in land_use_values if v > 0]
        if land_use_values:
            entropy = -np.sum([p * np.log2(p) for p in land_use_values])
            max_entropy = np.log2(len(land_use_values)) if len(land_use_values) > 0 else 1
            land_use_diversity = entropy / max_entropy if max_entropy > 0 else 0
        else:
            land_use_diversity = 0
        
        ndvi_normalized = ((row['ndvi_mean'] - ndvi_min) / (ndvi_max - ndvi_min + 1e-6))
        tree_normalized = ((row['tree_count'] - tree_min) / (tree_max - tree_min + 1e-6))
        open_area = row.get('land_use_city_open_area_percent', 0.0) / 100
        green_space_score = (ndvi_normalized + tree_normalized + open_area) / 3
        
        transit_raw = (row['transit_count'] - transit_min) / (transit_max - transit_min + 1e-6)
        transit_score = np.log1p(transit_raw * 10) / np.log1p(10)
        
        intersection_density = row.get('intersection_density', 0.0)
        intersection_density_normalized = (intersection_density - intersection_density_min) / (intersection_density_max - intersection_density_min + 1e-6)
        road_connectivity = np.log1p(intersection_density_normalized * 10) / np.log1p(10)
        
        # Hybrid safety score: combine accident_count and avg_road_accident_density
        accident_count_density = row['accident_count'] / row['area_km2'] if row['area_km2'] > 0 else 0
        accident_count_density_max = (accident_count_max / neighborhoods_df['area_km2'].min()) if neighborhoods_df['area_km2'].min() > 0 else 1
        accident_count_density = min(accident_count_density, accident_count_density_max * 0.5)  # Cap to reduce urban bias
        safety_score_count = 1 - (accident_count_density / (accident_count_density_max + 1e-6))
        
        accident_density = row.get('avg_road_accident_density', 0.0)
        safety_score_roads = 1 - (accident_density / (accident_density_max + 1e-6))
        
        # Combine both safety metrics (50% weight each)
        safety_score = 0.5 * safety_score_count + 0.5 * safety_score_roads
        
        elderly_accessibility = row['elderly_percentage'] / 100
        
        base_score = (
            0.3 * land_use_diversity +
            0.3 * green_space_score +
            0.2 * transit_score +
            0.2 * road_connectivity
        )
        
        safety_modifier = 0.3 + 0.7 * safety_score  # Increased weight for safety
        elderly_modifier = 1 + elderly_accessibility
        walkability_score = base_score * safety_modifier * elderly_modifier
        
        walkability_score = 1 / (1 + np.exp(-5 * (walkability_score - 1)))
        
        components['LIE_NAME'].append(row['LIE_NAME'])
        components['land_use_diversity'].append(land_use_diversity)
        components['green_space_score'].append(green_space_score)
        components['transit_score'].append(transit_score)
        components['road_connectivity'].append(road_connectivity)
        components['safety_score'].append(safety_score)
        components['elderly_accessibility'].append(elderly_accessibility)
        components['walkability_score'].append(walkability_score)
    
    walkability_df = pd.DataFrame(components)
    print("Walkability score distribution:")
    print(walkability_df['walkability_score'].describe())
    
    corr, p_value = pearsonr(walkability_df['walkability_score'], neighborhoods_df['transit_count'])
    logging.info(f"Correlation between walkability score and transit count: {corr:.2f} (p-value: {p_value:.2f})")
    
    return walkability_df

Cell 4 Main Data Loading and Processing

In [92]:
def load_and_prepare_data():
    logging.info("Stage 1: Loading and preparing data...")
    with tqdm(total=8, desc="Loading files") as pbar:
        neighborhoods_gdf = gpd.read_file(
            LANDUSE_NDVI_PATH,
            encoding='utf-8-sig',
            columns=['LIE_NAME', 'geometry', 'land_use_residential_percent', 'land_use_commercial_percent',
                     'land_use_education_percent', 'ndvi_mean']
        ).to_crs('EPSG:3826')
        total_rows = len(neighborhoods_gdf)
        unique_names = neighborhoods_gdf['LIE_NAME'].nunique()
        duplicates = neighborhoods_gdf['LIE_NAME'].duplicated().sum()
        logging.info(f"Loaded {total_rows} neighborhoods (expected 456)")
        logging.info(f"Unique LIE_NAME values: {unique_names}")
        if duplicates > 0:
            logging.warning(f"Found {duplicates} duplicate LIE_NAME values:")
            duplicate_names = neighborhoods_gdf[neighborhoods_gdf['LIE_NAME'].duplicated(keep=False)][['LIE_NAME']].reset_index()
            logging.info(f"Duplicate LIE_NAME details:\n{duplicate_names.to_string(index=False)}")
            raise ValueError("Duplicate LIE_NAME values detected. Please correct the data.")
        if unique_names != 456:
            logging.error(f"Expected 456 unique LIE_NAME values, found {unique_names}.")
            raise ValueError("LIE_NAME count mismatch.")
        neighborhoods_gdf = neighborhoods_gdf.reset_index(drop=True)
        pbar.update(1)

        buildings_gdf = gpd.read_file(OSM_BUILDINGS_PATH, columns=['geometry', 'building']).to_crs('EPSG:3826')
        buildings_gdf['area_m2'] = buildings_gdf.geometry.area
        pbar.update(1)

        roads_gdf = gpd.read_parquet(OSM_ROADS_PATH, columns=['geometry', 'class']).to_crs('EPSG:3826')
        roads_gdf['length_m'] = roads_gdf.geometry.length
        logging.info(f"Found {roads_gdf['class'].isnull().sum()} roads with missing 'class' values.")
        roads_gdf['class'] = roads_gdf['class'].fillna('unknown')
        pbar.update(1)

        trees_gdf = gpd.read_parquet(OSM_TREES_PATH, columns=['geometry', 'subtype', 'class']).to_crs('EPSG:3826')
        trees_gdf = trees_gdf[trees_gdf['subtype'] == 'tree']
        pbar.update(1)

        transit_gdf = gpd.read_parquet(OSM_TRANSIT_PATH, columns=['geometry', 'class']).to_crs('EPSG:3826')
        transit_gdf = transit_gdf[transit_gdf['class'].isin(['stop_position', 'bus_stop'])]
        pbar.update(1)

        urban_masterplan_gdf = gpd.read_file(URBAN_MASTERPLAN_PATH, columns=['geometry', 'Category']).to_crs('EPSG:3826')
        if 'area' not in urban_masterplan_gdf.columns:
            urban_masterplan_gdf['area'] = urban_masterplan_gdf.geometry.area
        pbar.update(1)

        # Load accidents with severity if available
        try:
            accidents_gdf = gpd.read_file(ACCIDENTS_PATH, columns=['geometry', 'severity']).to_crs('EPSG:3826')
            logging.info("Loaded accidents with severity column.")
        except ValueError:
            accidents_gdf = gpd.read_file(ACCIDENTS_PATH, columns=['geometry']).to_crs('EPSG:3826')
            logging.warning("No severity column in accidents data. Using count only.")
        pbar.update(1)

        population_df = pd.read_json(POPULATION_PATH, encoding='utf-8')
        population_df.rename(columns={'District': 'LIE_NAME', 'Total_Population': 'total_population',
                                      'Elderly_Percentage': 'elderly_percentage'}, inplace=True)
        pbar.update(1)

    logging.info("Validating and fixing geometries...")
    neighborhoods_gdf['geometry'] = neighborhoods_gdf['geometry'].apply(fix_geometry)
    urban_masterplan_gdf['geometry'] = urban_masterplan_gdf['geometry'].apply(fix_geometry)
    roads_gdf['geometry'] = roads_gdf['geometry'].apply(fix_geometry)
    accidents_gdf['geometry'] = accidents_gdf['geometry'].apply(fix_geometry)

    invalid_neighborhoods = neighborhoods_gdf[~neighborhoods_gdf.geometry.is_valid]
    if not invalid_neighborhoods.empty:
        logging.warning(f"Found {len(invalid_neighborhoods)} invalid geometries in neighborhoods_gdf.")
    invalid_masterplan = urban_masterplan_gdf[~urban_masterplan_gdf.geometry.is_valid]
    if not invalid_masterplan.empty:
        logging.warning(f"Found {len(invalid_masterplan)} invalid geometries in urban_masterplan_gdf.")
    invalid_roads = roads_gdf[~roads_gdf.geometry.is_valid]
    if not invalid_roads.empty:
        logging.warning(f"Found {len(invalid_roads)} invalid geometries in roads_gdf.")
    invalid_accidents = accidents_gdf[~accidents_gdf.geometry.is_valid]
    if not invalid_accidents.empty:
        logging.warning(f"Found {len(invalid_accidents)} invalid geometries in accidents_gdf.")

    logging.info("Performing spatial joins and aggregations...")
    neighborhoods_gdf['area_km2'] = neighborhoods_gdf.geometry.area / 1e6

    # Compute accident_count with border sharing
    logging.info("Computing accident counts per neighborhood with border sharing...")
    # Step 1: Create a buffered version of neighborhood boundaries
    buffer_distance = 10  # meters, adjustable
    neighborhoods_buffered = neighborhoods_gdf.copy()
    neighborhoods_buffered['geometry'] = neighborhoods_buffered['geometry'].boundary.buffer(buffer_distance)
    
    # Step 2: Assign accidents to neighborhoods (initial assignment)
    accident_counts = gpd.sjoin(neighborhoods_gdf, accidents_gdf, how='left', predicate='contains')
    accident_counts = accident_counts.groupby(level=0).size().reindex(neighborhoods_gdf.index, fill_value=0)
    neighborhoods_gdf['accident_count'] = accident_counts
    
    # Step 3: Identify accidents near borders and share them
    accidents_near_borders = gpd.sjoin(accidents_gdf, neighborhoods_buffered[['geometry', 'LIE_NAME']], how='left', predicate='intersects')
    # Group by accident to see how many neighborhoods are near each accident
    accidents_near_borders['weight'] = accidents_near_borders.groupby(accidents_near_borders.index).transform('size')
    accidents_near_borders['weight'] = 1 / accidents_near_borders['weight']  # Equal sharing
    # Aggregate shared counts
    shared_counts = accidents_near_borders.groupby('LIE_NAME')['weight'].sum().reindex(neighborhoods_gdf['LIE_NAME'], fill_value=0)
    neighborhoods_gdf['accident_count'] = neighborhoods_gdf['accident_count'] + shared_counts
    neighborhoods_gdf['accident_count'] = neighborhoods_gdf['accident_count'].round().astype(int)

    tree_counts = gpd.sjoin(neighborhoods_gdf, trees_gdf, how='left', predicate='contains')
    tree_counts = tree_counts.groupby(level=0).size().reindex(neighborhoods_gdf.index, fill_value=0)
    neighborhoods_gdf['tree_count'] = tree_counts

    transit_counts = gpd.sjoin(neighborhoods_gdf, transit_gdf, how='left', predicate='contains')
    transit_counts = transit_counts.groupby(level=0).size().reindex(neighborhoods_gdf.index, fill_value=0)
    neighborhoods_gdf['transit_count'] = transit_counts

    road_lengths = gpd.sjoin(roads_gdf, neighborhoods_gdf, how='left', predicate='intersects')
    road_lengths = road_lengths.groupby('index_right')['length_m'].sum().reindex(neighborhoods_gdf.index, fill_value=0)
    neighborhoods_gdf['road_density'] = road_lengths / (neighborhoods_gdf['area_km2'] * 1000)

    logging.info("Computing intersection counts...")
    neighborhoods_gdf = compute_intersection_counts(neighborhoods_gdf, roads_gdf)

    neighborhoods_gdf = neighborhoods_gdf.merge(
        population_df[['LIE_NAME', 'total_population', 'elderly_percentage']],
        on='LIE_NAME',
        how='left'
    )
    neighborhoods_gdf['total_population'] = neighborhoods_gdf['total_population'].fillna(0)
    neighborhoods_gdf['elderly_percentage'] = neighborhoods_gdf['elderly_percentage'].fillna(0)

    logging.info("Computing land use percentages...")
    temp_gdf = gpd.GeoDataFrame({'geometry': neighborhoods_gdf['geometry']}, crs='EPSG:3826')
    intersected = gpd.overlay(temp_gdf, urban_masterplan_gdf, how='intersection', keep_geom_type=False)
    intersected['geometry'] = intersected['geometry'].apply(fix_geometry)
    intersected = intersected[intersected.geometry.is_valid & ~intersected.geometry.is_empty]
    
    if not intersected.empty:
        intersected['priority'] = intersected['Category'].map(CATEGORY_PRIORITY)
        intersected = intersected.sort_values(by='priority', ascending=False)
        intersected['area'] = intersected.geometry.area
        
        for category in CATEGORY_PRIORITY.keys():
            col_name = f"land_use_{category.lower()}_percent"
            category_areas = intersected[intersected['Category'] == category].groupby(level=0)['area'].sum()
            total_areas = intersected.groupby(level=0)['area'].sum()
            percentages = (category_areas / total_areas * 100).reindex(neighborhoods_gdf.index, fill_value=0.0)
            neighborhoods_gdf[col_name] = percentages
    else:
        for category in CATEGORY_PRIORITY.keys():
            col_name = f"land_use_{category.lower()}_percent"
            neighborhoods_gdf[col_name] = 0.0

    # Clustering-based imputation for missing land use percentages
    logging.info("Imputing missing land use percentages using clustering...")
    from sklearn.cluster import KMeans
    cluster_features = neighborhoods_gdf[['ndvi_mean', 'total_population']].fillna(0)
    kmeans = KMeans(n_clusters=5, random_state=42).fit(cluster_features)
    neighborhoods_gdf['cluster'] = kmeans.labels_
    for category in CATEGORY_PRIORITY.keys():
        col_name = f"land_use_{category.lower()}_percent"
        medians = neighborhoods_gdf.groupby('cluster')[col_name].median().fillna(0)
        neighborhoods_gdf[col_name] = neighborhoods_gdf.apply(
            lambda row: medians[row['cluster']] if pd.isna(row[col_name]) or row[col_name] == 0.0 else row[col_name], axis=1)
    neighborhoods_gdf.drop('cluster', axis=1, inplace=True)
    logging.info("Clustering-based imputation completed.")

    print_data_structure({
        'neighborhoods': neighborhoods_gdf,
        'buildings': buildings_gdf,
        'roads': roads_gdf,
        'trees': trees_gdf,
        'transit': transit_gdf,
        'urban_masterplan': urban_masterplan_gdf,
        'accidents': accidents_gdf,
        'population': population_df
    })

    return {
        'neighborhoods': neighborhoods_gdf,
        'buildings': buildings_gdf,
        'roads': roads_gdf,
        'trees': trees_gdf,
        'transit': transit_gdf,
        'urban_masterplan': urban_masterplan_gdf,
        'accidents': accidents_gdf,
        'population': population_df
    }

Cell 5 compute_intersection_counts

In [93]:
def compute_intersection_counts(neighborhoods_gdf, roads_gdf):
    logging.info("Computing intersection counts per neighborhood...")
    if CUSPATIAL_AVAILABLE:
        logging.info("Using cuspatial for intersection counts...")
        try:
            road_points = roads_gdf.geometry.apply(lambda x: x.boundary).explode(index_parts=True)
            road_points = road_points[road_points.geom_type == 'Point']
            points = cuspatial.GeoSeries(road_points)
            polys = cuspatial.GeoSeries(neighborhoods_gdf.geometry)
            hits = cuspatial.points_in_polygon(points, polys)
            intersection_counts = hits.sum(axis=1).reindex(neighborhoods_gdf.index, fill_value=0)
            neighborhoods_gdf['intersection_count'] = intersection_counts
        except Exception as e:
            logging.warning(f"CuSpatial failed: {e}. Falling back to geopandas...")
            endpoint_to_roads = {}
            for road_idx, geom in tqdm(roads_gdf.geometry.items(), total=len(roads_gdf), desc="Building endpoint-to-road mapping WIP"):
                if geom.geom_type == 'LineString':
                    start = geom.coords[0]
                    end = geom.coords[-1]
                    endpoint_to_roads.setdefault(start, []).append(road_idx)
                    endpoint_to_roads.setdefault(end, []).append(road_idx)
                elif geom.geom_type == 'MultiLineString':
                    for line in geom.geoms:
                        start = line.coords[0]
                        end = line.coords[-1]
                        endpoint_to_roads.setdefault(start, []).append(road_idx)
                        endpoint_to_roads.setdefault(end, []).append(road_idx)
            
            intersections = []
            for node in tqdm(endpoint_to_roads.keys(), desc="Identifying intersections"):
                if len(endpoint_to_roads[node]) > 2:
                    intersections.append(Point(node))
            
            intersections_gdf = gpd.GeoDataFrame(geometry=intersections, crs='EPSG:3826')
            intersection_counts = gpd.sjoin(neighborhoods_gdf, intersections_gdf, how='left', predicate='contains')
            intersection_counts = intersection_counts.groupby(level=0).size().reindex(neighborhoods_gdf.index, fill_value=0)
            neighborhoods_gdf['intersection_count'] = intersection_counts
    else:
        logging.info("Using geopandas for intersection counts...")
        endpoint_to_roads = {}
        for road_idx, geom in tqdm(roads_gdf.geometry.items(), total=len(roads_gdf), desc="Building endpoint-to-road mapping"):
            if geom.geom_type == 'LineString':
                start = geom.coords[0]
                end = geom.coords[-1]
                endpoint_to_roads.setdefault(start, []).append(road_idx)
                endpoint_to_roads.setdefault(end, []).append(road_idx)
            elif geom.geom_type == 'MultiLineString':
                for line in geom.geoms:
                    start = line.coords[0]
                    end = line.coords[-1]
                    endpoint_to_roads.setdefault(start, []).append(road_idx)
                    endpoint_to_roads.setdefault(end, []).append(road_idx)
        
        intersections = []
        for node in tqdm(endpoint_to_roads.keys(), desc="Identifying intersections"):
            if len(endpoint_to_roads[node]) > 2:
                intersections.append(Point(node))
        
        intersections_gdf = gpd.GeoDataFrame(geometry=intersections, crs='EPSG:3826')
        intersection_counts = gpd.sjoin(neighborhoods_gdf, intersections_gdf, how='left', predicate='contains')
        intersection_counts = intersection_counts.groupby(level=0).size().reindex(neighborhoods_gdf.index, fill_value=0)
        neighborhoods_gdf['intersection_count'] = intersection_counts

    neighborhoods_gdf['intersection_density'] = neighborhoods_gdf['intersection_count'] / neighborhoods_gdf['area_km2']
    neighborhoods_gdf['intersection_density'] = neighborhoods_gdf['intersection_density'].fillna(0.0)
    logging.info("Intersection counts completed.")
    return neighborhoods_gdf

Cell 6: Graph Construction (build_graph)

In [94]:
def build_graph(data, force_recompute=False):
    logging.info("Stage 2: Building city graph...")
    G = cugraph.Graph(directed=False)
    
    current_hash = compute_data_hash(data)
    
    if not force_recompute and os.path.exists(GRAPH_NODES_CACHE_PATH) and os.path.exists(GRAPH_EDGES_CACHE_PATH) and os.path.exists(GRAPH_DATA_HASH_PATH):
        with open(GRAPH_DATA_HASH_PATH, 'r') as f:
            cached_hash = f.read().strip()
        if cached_hash == current_hash:
            logging.info("Loading graph from cache...")
            try:
                nodes_df = cudf.read_parquet(GRAPH_NODES_CACHE_PATH)
                edges_df = cudf.read_parquet(GRAPH_EDGES_CACHE_PATH)
                G._nodes = nodes_df
                if not edges_df.empty:
                    G.from_cudf_edgelist(edges_df, source='src', destination='dst')
                logging.info(f"City graph loaded from cache: {len(nodes_df)} nodes, {len(edges_df)} edges")
                return G
            except Exception as e:
                logging.warning(f"Failed to load cached graph: {e}. Recomputing...")
    
    nodes = []
    edges = []
    node_id_to_index = {}
    current_idx = 0
    
    logging.info("Adding neighborhood nodes...")
    neighborhoods_gdf = data['neighborhoods']
    logging.info(f"Processing {len(neighborhoods_gdf)} neighborhoods (expected 456)")
    if len(neighborhoods_gdf) != 456 or neighborhoods_gdf['LIE_NAME'].nunique() != 456:
        logging.error(f"Neighborhood count mismatch: {len(neighborhoods_gdf)} rows, {neighborhoods_gdf['LIE_NAME'].nunique()} unique names.")
        raise ValueError("Expected 456 unique neighborhoods.")
    for idx, row in neighborhoods_gdf.iterrows():
        node_id = f"neighborhood_{idx}"
        node_data = {
            'vertex': node_id,
            'type': 'neighborhood',
            'LIE_NAME': row['LIE_NAME'],
            'ndvi_mean': row['ndvi_mean'],
            'tree_count': row['tree_count'],
            'transit_count': row['transit_count'],
            'accident_count': row['accident_count'],
            'road_density': row['road_density'],
            'intersection_count': row.get('intersection_count', 0),
            'intersection_density': row.get('intersection_density', 0.0),
            'total_population': row['total_population'],
            'elderly_percentage': row['elderly_percentage'],
            'area_km2': row['area_km2']
        }
        for category in CATEGORY_PRIORITY.keys():
            col_name = f"land_use_{category.lower()}_percent"
            node_data[col_name] = row.get(col_name, 0.0)
        
        nodes.append(node_data)
        node_id_to_index[node_id] = current_idx
        current_idx += 1
    logging.info(f"Added {len(nodes)} neighborhood nodes")
    
    logging.info("Adding building nodes...")
    buildings_gdf = data['buildings']
    for idx, row in buildings_gdf.iterrows():
        node_id = f"building_{idx}"
        node_data = {
            'vertex': node_id,
            'type': 'building',
            'building': row['building'] if pd.notna(row['building']) else 'unknown',
            'area_m2': row['area_m2']
        }
        nodes.append(node_data)
        node_id_to_index[node_id] = current_idx
        current_idx += 1
    
    logging.info("Adding road nodes...")
    roads_gdf = data['roads']
    for idx, row in roads_gdf.iterrows():
        node_id = f"road_{idx}"
        node_data = {
            'vertex': node_id,
            'type': 'road',
            'class': row['class'],
            'length_m': row['length_m']
        }
        nodes.append(node_data)
        node_id_to_index[node_id] = current_idx
        current_idx += 1
    
    nodes_df = cudf.DataFrame(nodes)
    logging.info(f"Created {len(nodes_df)} nodes, including {len(nodes_df[nodes_df['type'] == 'neighborhood'])} neighborhoods")
    
    logging.info("Building edges based on spatial proximity...")
    neighborhoods_gdf = neighborhoods_gdf.copy()
    buildings_gdf = buildings_gdf.copy()
    roads_gdf = data['roads'].copy()
    
    neighborhoods_gdf['geometry'] = neighborhoods_gdf['geometry'].apply(fix_geometry)
    buildings_gdf['geometry'] = buildings_gdf['geometry'].apply(fix_geometry)
    roads_gdf['geometry'] = roads_gdf['geometry'].apply(fix_geometry)
    
    neighborhood_sindex = neighborhoods_gdf.sindex
    building_sindex = buildings_gdf.sindex
    road_sindex = roads_gdf.sindex
    
    logging.info("Computing neighborhood-neighborhood edges...")
    for i, row_i in tqdm(neighborhoods_gdf.iterrows(), total=len(neighborhoods_gdf), desc="Neighborhood-neighborhood edges"):
        node_i = f"neighborhood_{i}"
        geom_i = row_i['geometry']
        possible_matches = list(neighborhood_sindex.query(geom_i, predicate='touches'))
        for j in possible_matches:
            if i < j:
                node_j = f"neighborhood_{j}"
                geom_j = neighborhoods_gdf.iloc[j]['geometry']
                try:
                    if geom_i.touches(geom_j):
                        edges.append({'src': node_i, 'dst': node_j})
                except Exception as e:
                    logging.warning(f"Error checking touches between {node_i} and {node_j}: {e}")
    
    logging.info("Computing neighborhood-building edges...")
    if CUSPATIAL_AVAILABLE:
        logging.info("Using cuspatial for neighborhood-building edges...")
        building_points = buildings_gdf['geometry'].centroid
        building_cudf = cuspatial.GeoSeries(building_points)
        batch_size = 31  # cuSpatial limit
        for start_idx in tqdm(range(0, len(neighborhoods_gdf), batch_size), desc="Neighborhood-building edges (cuspatial)"):
            end_idx = min(start_idx + batch_size, len(neighborhoods_gdf))
            batch_gdf = neighborhoods_gdf.iloc[start_idx:end_idx]
            poly = cuspatial.GeoSeries(batch_gdf['geometry'])
            try:
                hits = cuspatial.point_in_polygon(building_cudf, poly)
                for i, idx in enumerate(batch_gdf.index):
                    hit_indices = hits[hits[i]].index.to_pandas()
                    node_i = f"neighborhood_{idx}"
                    for j in hit_indices:
                        edges.append({'src': node_i, 'dst': f"building_{j}"})
            except Exception as e:
                logging.warning(f"cuspatial error for batch {start_idx}-{end_idx}: {e}. Falling back to geopandas...")
                for idx in batch_gdf.index:
                    node_i = f"neighborhood_{idx}"
                    geom_i = batch_gdf.loc[idx, 'geometry']
                    possible_matches = list(building_sindex.query(geom_i, predicate='contains'))
                    for j in possible_matches:
                        node_j = f"building_{j}"
                        geom_j = buildings_gdf.iloc[j]['geometry']
                        try:
                            if geom_i.contains(geom_j):
                                edges.append({'src': node_i, 'dst': node_j})
                        except Exception as e:
                            logging.warning(f"Error checking containment between {node_i} and {node_j}: {e}")
    else:
        logging.info("Using geopandas for neighborhood-building edges...")
        for i, row_i in tqdm(neighborhoods_gdf.iterrows(), total=len(neighborhoods_gdf), desc="Neighborhood-building edges"):
            node_i = f"neighborhood_{i}"
            geom_i = row_i['geometry']
            possible_matches = list(building_sindex.query(geom_i, predicate='contains'))
            for j in possible_matches:
                node_j = f"building_{j}"
                geom_j = buildings_gdf.iloc[j]['geometry']
                try:
                    if geom_i.contains(geom_j):
                        edges.append({'src': node_i, 'dst': node_j})
                except Exception as e:
                    logging.warning(f"Error checking containment between {node_i} and {node_j}: {e}")
    
    logging.info("Computing neighborhood-road edges...")
    for i, row_i in tqdm(neighborhoods_gdf.iterrows(), total=len(neighborhoods_gdf), desc="Neighborhood-road edges"):
        node_i = f"neighborhood_{i}"
        geom_i = row_i['geometry']
        possible_matches = list(road_sindex.query(geom_i, predicate='intersects'))
        for j in possible_matches:
            node_j = f"road_{j}"
            geom_j = roads_gdf.iloc[j]['geometry']
            try:
                if geom_i.intersects(geom_j):
                    edges.append({'src': node_i, 'dst': node_j})
            except Exception as e:
                logging.warning(f"Error checking intersection between {node_i} and {node_j}: {e}")
    
    edges_df = cudf.DataFrame(edges)
    
    valid_nodes = set(nodes_df['vertex'].to_pandas())
    edges_df = edges_df[edges_df['src'].isin(valid_nodes) & edges_df['dst'].isin(valid_nodes)]
    
    G._nodes = nodes_df
    if not edges_df.empty:
        G.from_cudf_edgelist(edges_df, source='src', destination='dst')
    else:
        logging.warning("No valid edges created. Graph will have nodes but no edges.")
    
    logging.info("Saving graph data to cache...")
    try:
        nodes_df.to_parquet(GRAPH_NODES_CACHE_PATH)
        edges_df.to_parquet(GRAPH_EDGES_CACHE_PATH)
        with open(GRAPH_DATA_HASH_PATH, 'w') as f:
            f.write(current_hash)
        with open(GRAPH_NODE_ID_CACHE_PATH, 'w') as f:
            json.dump(node_id_to_index, f)
        logging.info("Successfully saved graph data to cache.")
    except Exception as e:
        logging.error(f"Failed to save graph data to cache: {e}")
    
    logging.info(f"City graph constructed: {len(nodes_df)} nodes, {len(edges_df)} edges")
    return G

Cell 7: Rule-Based Walkability Scores (compute_walkability_scores)

In [95]:
def compute_walkability_scores(G):
    logging.info("Stage 3: Calculating rule-based walkability scores...")
    nodes_df = G._nodes.to_pandas()
    
    neighborhood_nodes = nodes_df[nodes_df['type'] == 'neighborhood']
    
    walkability_components = compute_walkability_components_all(neighborhood_nodes)
    
    nodes_df['walkability_score'] = np.nan
    
    lie_name_to_score = {}
    for _, row in walkability_components.iterrows():
        lie_name = row['LIE_NAME']
        score = row['walkability_score']
        lie_name_to_score.setdefault(lie_name, []).append(score)
    
    for lie_name, scores in lie_name_to_score.items():
        mask = nodes_df['LIE_NAME'] == lie_name
        if mask.any():
            nodes_df.loc[mask, 'walkability_score'] = np.mean(scores)
        else:
            logging.warning(f"No node found for LIE_NAME {lie_name} when assigning walkability score.")
    
    G._nodes = cudf.from_pandas(nodes_df)
    
    logging.info("Rule-based walkability scores calculated and added to graph.")
    return G

Cell 8:Subgraph Functionality

Cell 9 prepare_gnn_data

In [96]:
def prepare_gnn_data(G):
    logging.info("Preparing GNN data...")
    nodes_df = G._nodes.to_pandas()
    edges_df = G.edgelist.edgelist_df.to_pandas()
    
    try:
        with open(GRAPH_NODE_ID_CACHE_PATH, 'r') as f:
            node_id_to_index = json.load(f)
    except Exception as e:
        logging.warning(f"Failed to load node_id_to_index: {e}. Rebuilding mapping...")
        node_id_to_index = {row['vertex']: idx for idx, row in nodes_df.iterrows()}
    
    feature_cols = [
        'land_use_residential_percent', 'land_use_commercial_percent', 'land_use_education_percent',
        'land_use_city_open_area_percent', 'land_use_public_transportation_percent', 'land_use_pedestrian_percent',
        'ndvi_mean', 'tree_count', 'transit_count', 'accident_count', 'road_density', 'intersection_density',
        'total_population', 'elderly_percentage'
    ]
    
    for col in feature_cols:
        if col not in nodes_df.columns:
            nodes_df[col] = 0.0
        else:
            nodes_df[col] = nodes_df[col].fillna(0.0)
    
    for col in feature_cols:
        if nodes_df[col].std() > 0:
            nodes_df[col] = (nodes_df[col] - nodes_df[col].mean()) / nodes_df[col].std()
    
    feature_matrix = nodes_df[feature_cols].values
    feature_matrix = np.nan_to_num(feature_matrix, nan=0.0)
    
    edges_df['src_idx'] = edges_df['src'].map(node_id_to_index)
    edges_df['dst_idx'] = edges_df['dst'].map(node_id_to_index)
    
    edges_df = edges_df.dropna(subset=['src_idx', 'dst_idx'])
    
    edges_df['src_idx'] = edges_df['src_idx'].astype(int)
    edges_df['dst_idx'] = edges_df['dst_idx'].astype(int)
    
    edge_index = torch.tensor(
        [edges_df['src_idx'].values, edges_df['dst_idx'].values],
        dtype=torch.long
    )
    
    edge_attr = None
    if 'weight' in edges_df.columns:
        edge_attr = torch.tensor(edges_df['weight'].values, dtype=torch.float)
    
    y = torch.tensor(nodes_df['walkability_score'].fillna(0.0).values, dtype=torch.float)
    
    node_type_mapping = {
        'neighborhood': 0,
        'building': 1,
        'road': 2,
        'tree': 3,
        'transit': 4
    }
    node_type = nodes_df['type'].map(node_type_mapping).fillna(-1).astype(int).values
    node_type = torch.tensor(node_type, dtype=torch.long)
    
    data = Data(
        x=torch.tensor(feature_matrix, dtype=torch.float),
        edge_index=edge_index,
        edge_attr=edge_attr,
        y=y,
        node_type=node_type
    )
    
    logging.info("GNN data prepared.")
    return data

Cell 10: WalkabilityGNN, train_gnn_model, predict_walkability

In [97]:
class WalkabilityGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super(WalkabilityGNN, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1)
    
    def forward(self, x, edge_index, edge_attr=None):
        x = self.conv1(x, edge_index, edge_attr).relu()
        x = self.conv2(x, edge_index, edge_attr)
        return x

def train_gnn_model(data):
    logging.info("Stage 4: Training GNN model...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = WalkabilityGNN(
        in_channels=data.x.shape[1],
        hidden_channels=128,
        out_channels=1,
        heads=4
    ).to(device)
    data = data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

    model.train()
    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_attr).squeeze()
        mask = data.node_type == 0
        loss = F.mse_loss(out[mask], data.y[mask])
        loss.backward()
        optimizer.step()
        scheduler.step(loss)
        if epoch % 10 == 0:
            logging.info(f"Epoch {epoch}, Loss: {loss.item():.4f}")

    model.eval()
    logging.info("Finished training GNN model.")
    return model

def predict_walkability(G, model):
    logging.info("Predicting walkability with GNN...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    data = prepare_gnn_data(G)
    data = data.to(device)

    with torch.no_grad():
        predictions = model(data.x, data.edge_index, data.edge_attr).squeeze()

    predictions = predictions.cpu().numpy()
    predictions = 1 / (1 + np.exp(-predictions))

    nodes_df = G._nodes.to_pandas()
    nodes_df['walkability_gnn'] = predictions
    G._nodes = cudf.from_pandas(nodes_df)

    logging.info("Finished predicting walkability with GNN.")
    return G

Cell 11: Interactive Map Generation (create_interactive_map)

In [98]:
def create_interactive_map(G, data):
    logging.info("Generating interactive Kepler.gl map...")
    nodes_df = G._nodes.to_pandas()
    neighborhoods_gdf = data['neighborhoods'].copy()

    nodes_df['LIE_NAME'] = nodes_df['LIE_NAME'].astype(str).str.strip()
    neighborhoods_gdf['LIE_NAME'] = neighborhoods_gdf['LIE_NAME'].astype(str).str.strip()

    neighborhood_nodes = nodes_df[nodes_df['type'] == 'neighborhood'].copy()

    nodes_lie_names = set(neighborhood_nodes['LIE_NAME'])
    gdf_lie_names = set(neighborhoods_gdf['LIE_NAME'])
    logging.info(f"Neighborhood nodes count: {len(neighborhood_nodes)}")
    logging.info(f"Neighborhoods_gdf count: {len(neighborhoods_gdf)}")
    logging.info(f"Sample LIE_NAME in nodes_df: {list(nodes_lie_names)[:5]}")
    logging.info(f"Sample LIE_NAME in neighborhoods_gdf: {list(gdf_lie_names)[:5]}")
    logging.info(f"Common LIE_NAMEs: {len(nodes_lie_names & gdf_lie_names)}")
    logging.info(f"Nodes LIE_NAMEs not in GDF: {list(nodes_lie_names - gdf_lie_names)}")
    logging.info(f"GDF LIE_NAMEs not in nodes: {list(gdf_lie_names - nodes_lie_names)}")
    logging.info(f"Nodes nulls: {neighborhood_nodes.isna().sum().to_dict()}")
    logging.info(f"GDF geometry nulls: {neighborhoods_gdf['geometry'].isna().sum()}")

    map_data = neighborhoods_gdf[['LIE_NAME', 'geometry']].merge(
        neighborhood_nodes[['LIE_NAME', 'walkability_score', 'walkability_gnn']],
        on='LIE_NAME',
        how='left'
    )

    map_data = map_data.drop_duplicates(subset=['LIE_NAME'], keep='first')

    logging.info(f"Merged map_data rows: {len(map_data)}")
    logging.info(f"Walkability score nulls: {map_data['walkability_score'].isna().sum()}")
    logging.info(f"Walkability GNN nulls: {map_data['walkability_gnn'].isna().sum()}")

    map_data['walkability_score'] = map_data['walkability_score'].fillna(0)
    map_data['walkability_gnn'] = map_data['walkability_gnn'].fillna(0)

    map_data = gpd.GeoDataFrame(map_data, geometry='geometry', crs='EPSG:3826')

    map_data['geometry'] = map_data['geometry'].to_crs('EPSG:4326')
    kepler_data = {
        'neighborhoods': map_data[['LIE_NAME', 'walkability_score', 'walkability_gnn', 'geometry']].to_json()
    }

    config = {
        "version": "v1",
        "config": {
            "visState": {
                "layers": [
                    {
                        "id": "neighborhoods",
                        "type": "geojson",
                        "config": {
                            "dataId": "neighborhoods",
                            "label": "Neighborhoods",
                            "color": [18, 147, 154],
                            "columns": {
                                "geojson": "geometry"
                            },
                            "isVisible": True,
                            "visConfig": {
                                "opacity": 0.7,
                                "strokeOpacity": 0.9,
                                "thickness": 1,
                                "strokeColor": [255, 255, 255],
                                "colorRange": {
                                    "name": "Global Warming",
                                    "type": "sequential",
                                    "colors": [
                                        "#5A1846", "#900C3F", "#C70039",
                                        "#E3611C", "#F1920E", "#FFC107"
                                    ]
                                },
                                "strokeColorRange": {
                                    "name": "Global Warming",
                                    "type": "sequential",
                                    "colors": [
                                        "#5A1846", "#900C3F", "#C70039",
                                        "#E3611C", "#F1920E", "#FFC107"
                                    ]
                                },
                                "colorField": {
                                    "name": "walkability_gnn",
                                    "type": "real"
                                },
                                "colorScale": "quantile"
                            }
                        },
                        "visualChannels": {
                            "colorField": {
                                "name": "walkability_gnn",
                                "type": "real"
                            },
                            "colorScale": "quantile"
                        }
                    }
                ],
                "interactionConfig": {
                    "tooltip": {
                        "fieldsToShow": {
                            "neighborhoods": [
                                {"name": "LIE_NAME", "format": None},
                                {"name": "walkability_score", "format": "{:.3f}"},
                                {"name": "walkability_gnn", "format": "{:.3f}"}
                            ]
                        },
                        "enabled": True
                    }
                }
            },
            "mapState": {
                "latitude": 25.0330,
                "longitude": 121.5654,
                "zoom": 11
            },
            "mapStyle": {
                "styleType": "dark"
            }
        }
    }

    map_1 = KeplerGl(height=800, data=kepler_data, config=config)
    map_path = os.path.join(BASE_DIR, 'taipei_walkability_map.html')
    map_1.save_to_html(file_name=map_path)
    logging.info(f"Interactive map generated and saved as {map_path}")
    print(f"Map saved to {map_path}!")

Cell 12: Main Execution (main)

In [99]:
def main(force_recompute_graph=False):
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    os.makedirs(SUBGRAPH_DIR, exist_ok=True)
    logging.info(f"Ensured subgraph directory exists: {SUBGRAPH_DIR}")

    print("Starting load_and_prepare_data...")
    data = load_and_prepare_data()
    print("Finished load_and_prepare_data")

    print("Starting compute_road_type_accident_correlation...")
    road_accident_summary = compute_road_type_accident_correlation(
        data['roads'], data['neighborhoods'])
    print("Finished compute_road_type_accident_correlation")

    print("Starting build_graph...")
    G = build_graph(data, force_recompute=force_recompute_graph)
    print("Finished build_graph")

    print("Starting compute_walkability_scores...")
    G = compute_walkability_scores(G)
    print("Finished compute_walkability_scores")

    print("Starting prepare_gnn_data...")
    data_gnn = prepare_gnn_data(G)
    print("Finished prepare_gnn_data")

    print("Starting train_gnn_model...")
    model = train_gnn_model(data_gnn)
    print("Finished train_gnn_model")

    print("Starting predict_walkability...")
    G = predict_walkability(G, model)
    print("Finished predict_walkability")

    print("Starting create_interactive_map...")
    create_interactive_map(G, data)
    print("Finished create_interactive_map")

    logging.info("Processing complete.")
    print(G.edgelist.edgelist_df.to_pandas().head())

if __name__ == "__main__":
    main(force_recompute_graph=True)

2025-04-17 18:14:31,494 - INFO - Ensured subgraph directory exists: /home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/subgraphs
2025-04-17 18:14:31,495 - INFO - Stage 1: Loading and preparing data...


Starting load_and_prepare_data...


Loading files:   0%|          | 0/8 [00:00<?, ?it/s]2025-04-17 18:14:31,581 - INFO - Loaded 456 neighborhoods (expected 456)
2025-04-17 18:14:31,581 - INFO - Unique LIE_NAME values: 456
Loading files: 100%|██████████| 8/8 [00:03<00:00,  2.19it/s]
2025-04-17 18:14:35,158 - INFO - Validating and fixing geometries...
2025-04-17 18:14:36,336 - INFO - Performing spatial joins and aggregations...
2025-04-17 18:14:36,337 - INFO - Computing accident counts per neighborhood with border sharing...


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer